In [ ]:
"""
Read an image & Brand Info -> convert to base64 -> send to openai for image detailed description ->
send description to creative model for creative ideas prompts -> send image, brand info, creative ideas prompts to creative model for design ->
covert image back from base64 -> Compress image to < 1MB & 24 Bit depth & Resize to 1080x1920 for TIKTOK ->save image -> save design prompt.


"""